<a href="https://colab.research.google.com/github/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set/blob/coords3/train_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install icecream
%tensorflow_version 1.x
import os
!pip install kora
from kora import drive
import time
!pip install colorama

ds_path='/content/drive/My Drive/PhD/datasets/cv_data/'

TensorFlow 1.x selected.


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
git_clone = True

if git_clone == True:
  os.chdir('/content')
  %rm -rf FCN_ConvLSTM_Crop_Recognition_Open_Set
  !git clone --branch coords3 https://github.com/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set.git

Cloning into 'FCN_ConvLSTM_Crop_Recognition_Open_Set'...
remote: Enumerating objects: 1747, done.
remote: Counting objects: 100% (1747/1747), done.
remote: Compressing objects: 100% (1243/1243), done.
remote: Total 1747 (delta 1121), reused 935 (delta 333), pack-reused 0
Receiving objects: 100% (1747/1747), 37.28 MiB | 28.53 MiB/s, done.
Resolving deltas: 100% (1121/1121), done.


## Download images into proper folder

In [4]:
!cp -r /content/drive/MyDrive/PhD/datasets/cv_data /content/FCN_ConvLSTM_Crop_Recognition_Open_Set/dataset/dataset/

In [5]:
os.chdir('/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src')
os.getcwd()
os.listdir()

['obj',
 'deb.py',
 'metrics.py',
 'model.py',
 'keras_weighted_categorical_crossentropy.py',
 'dataset.py',
 'model_best_UUnet4ConvLSTM_jun_cv_criteria_0_92',
 'model_input_mode.py',
 'generator.py',
 'densnet.py',
 'parameters',
 'monitor.py',
 'densnet_timedistributed.py',
 'patch_extractor.py',
 'analysis',
 '__init__.py',
 'main.py']

In [6]:
from colorama import init
init()
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Conv2DTranspose
# from keras.callbacks import ModelCheckpoint , EarlyStopping
from keras.optimizers import Adam,Adagrad 
from keras.models import Model
from keras import backend as K
import keras

import numpy as np
from sklearn.utils import shuffle
import cv2
import argparse
import tensorflow as tf

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import metrics
import sys
import glob

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
# Local
from densnet import DenseNetFCN
from densnet_timedistributed import DenseNetFCNTimeDistributed

#from metrics import fmeasure,categorical_accuracy
import deb
from keras_weighted_categorical_crossentropy import weighted_categorical_crossentropy, sparse_accuracy_ignoring_last_label, weighted_categorical_crossentropy_ignoring_last_label, categorical_focal_ignoring_last_label, weighted_categorical_focal_ignoring_last_label
from keras.models import load_model
from keras.layers import ConvLSTM2D, UpSampling2D, multiply
from keras.utils.vis_utils import plot_model
from keras.regularizers import l1,l2
import time
import pickle
#from keras_self_attention import SeqSelfAttention
import pdb
import pathlib
from pathlib import Path, PureWindowsPath
from keras.layers import Conv3DTranspose, Conv3D

from keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter


#from datagenerator import DataGenerator
from generator import DataGenerator, DataGeneratorWithCoords, DataGeneratorWithCoordsPatches

import matplotlib.pyplot as plt
sys.path.append('../../../dataset/dataset/patches_extract_script/')
from dataSource import DataSource, SARSource, OpticalSource, Dataset, LEM, LEM2, CampoVerde, OpticalSourceWithClouds, Humidity
from model_input_mode import MIMFixed, MIMVarLabel, MIMVarSeqLabel, MIMVarLabel_PaddedSeq, MIMFixedLabelAllLabels, MIMFixed_PaddedSeq
from parameters.parameters_reader import ParamsTrain

from icecream import ic
from monitor import Monitor, MonitorNPY, MonitorGenerator, MonitorNPYAndGenerator
import natsort
from model import NetModel, ModelFit, ModelLoadGenerator, ModelLoadGeneratorDebug, ModelLoadGeneratorWithCoords, ModelLoadEachBatch
from dataset import Dataset, DatasetWithCoords

from patch_extractor import PatchExtractor
ic.configureOutput(includeContext=False)
np.random.seed(2021)
tf.set_random_seed(2021)

from main import TrainTest

Using TensorFlow backend.


['parameters_openset_specifyunknownclasses.json', 'parameters_openset_lessclass8.json', 'no_mode.json', 'twokkc_parameters_closedset_groupclasses.json', 'twokkc_parameters_openset.json', 'allkkc_parameters_openset.json', 'cv', 'parameters_closedset_groupclasses_lessclass8.json', 'save_nonaugmented_train_patches_lessclass8.json', 'parameters_reader.py', 'parameters_openset.json', '__pycache__', 'parameters_closedset_groupclasses.json', 'twokkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches_unknownclasses.json', 'allkkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches.json', '__init__.py']


ic| self.seq_date: 'jun'


self.known_classes [1, 2, 6, 8]
[@debug] paramsTrain.seq_mode = fixed
[@debug] paramsTrain.mim = <model_input_mode.MIMFixed_PaddedSeq object at 0x7f80c6e93b50>


In [7]:
!nvidia-smi

Tue Jul 20 19:41:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:


paramsTrain = ParamsTrain('parameters/')
paramsTrain.mim = MIMFixed_PaddedSeq()

paramsTrain.dataSource = SARSource()

trainTest = TrainTest(paramsTrain)

patchExtractor = PatchExtractor(paramsTrain, trainTest.ds)	


['parameters_openset_specifyunknownclasses.json', 'parameters_openset_lessclass8.json', 'no_mode.json', 'twokkc_parameters_closedset_groupclasses.json', 'twokkc_parameters_openset.json', 'allkkc_parameters_openset.json', 'cv', 'parameters_closedset_groupclasses_lessclass8.json', 'save_nonaugmented_train_patches_lessclass8.json', 'parameters_reader.py', 'parameters_openset.json', '__pycache__', 'parameters_closedset_groupclasses.json', 'twokkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches_unknownclasses.json', 'allkkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches.json', '__init__.py']


ic| self.seq_date: 'jun'


self.known_classes [1, 2, 6, 8]
[@debug] self.ds = <dataSource.CampoVerde object at 0x7f80c10a6a90>
20151029
20151110
20151122
20151204
20151216
20160121
20160214
20160309
20160321
20160508
20160520
20160613
dotys_sin_cos.shape (12, 2)
[302, 314, 326, 338, 350, 21, 45, 69, 81, 129, 141, 165]
[[0.05084 0.7197 ]
 [0.1053  0.807  ]
 [0.1764  0.8813 ]
 [0.2612  0.9395 ]
 [0.3562  0.979  ]
 [0.6685  0.9707 ]
 [0.843   0.8643 ]
 [0.96    0.6963 ]
 [0.99    0.598  ]
 [0.905   0.2068 ]
 [0.8364  0.1301 ]
 [0.66    0.02637]]


ic| self.dataSource: <dataSource.SARSource object at 0x7f80c10a6a50>
ic| self.conf['path']/self.label_folder/"/": PosixPath('/')
ic| self.conf["in_npy_path"]: PosixPath('../../../dataset/dataset/cv_data/in_sar')
ic| self.conf["train"]["mask"]["dir"]: PosixPath('../../../dataset/dataset/cv_data/TrainTestMask.tif')
ic| os.getcwd(): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'


## Download or load sequence of images


In [9]:
if paramsTrain.getFullIms == True:
  patchExtractor.getFullIms()	
else:
  patchExtractor.fullImsLoad()


ic| patch["full_ims"].shape: (12, 8492, 7995, 2)
ic| self.dataset.im_list: ['20151029_S1',
                           '20151110_S1',
                           '20151122_S1',
                           '20151204_S1',
                           '20151216_S1',
                           '20160121_S1',
                           '20160214_S1',
                           '20160309_S1',
                           '20160321_S1',
                           '20160508_S1',
                           '20160520_S1',
                           '20160613_S1']


0 0
[@debug] conf["in_npy_path"]/(im_names[t_step]+".npy") = ../../../dataset/dataset/cv_data/in_sar/20151029_S1.npy
[@debug] patch["full_ims"].dtype = float16
[@debug] np.average(patch["full_ims"][t_step]) = 0.1871337890625
[@debug] np.max(patch["full_ims"][t_step]) = 1.0
[@debug] np.min(patch["full_ims"][t_step]) = 4.172325134277344e-07
../../../dataset/dataset/cv_data/labels/20151029_S1.tif
[@debug] conf["path"]/(self.dataSource.label_folder+"/"+label_names[t_step]+".tif") = ../../../dataset/dataset/cv_data/labels/20151029_S1.tif
[@debug] np.unique(patch["full_label_ims"][t_step],return_counts=True) = (array([ 0,  1,  2,  6,  7,  8,  9, 10, 11], dtype=int8), array([61778564,    45178,    51808,   131138,   438371,   155189,
        5136068,     1007,   156217]))
1 0
[@debug] conf["in_npy_path"]/(im_names[t_step]+".npy") = ../../../dataset/dataset/cv_data/in_sar/20151110_S1.npy
[@debug] patch["full_ims"].dtype = float16
[@debug] np.average(patch["full_ims"][t_step]) = 0.180419921875


ic| patch["full_ims"].shape: (12, 8492, 7995, 2)
ic| patch["full_label_ims"].shape: (12, 8492, 7995)
ic| patch["full_ims"].dtype: dtype('float16')
ic| patch["full_label_ims"].dtype: dtype('int8')


[@debug] np.unique(patch['full_label_ims'],return_counts=True) = (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int8), array([741342768,  17247080,   7809932,  12457060,    185790,     51702,
         2462898,   5372125,   1862268,  24044169,     12084,   1874604]))
0.0 1.0 0.1702
[@debug] self.dataset.name = cv
[@debug] self.dataset.scaler_name = cv
[@debug] self.dataset.seq_mode = fixed
[@debug] self.dataset.seq_date = jun
[@debug] self.dataset.scaler_load = False
[@debug] im.shape = (12, 8492, 7995, 2)
[@debug] im_flat[mask_flat==1,:].shape = (38664648, 2)
0.0001407 1.0 0.0779
[@debug] im_norm.shape = (12, 8492, 7995, 2)
FINISHED NORMALIZING, RESULT:
-1.432 51.53 3.117
[@debug] im.shape = (12, 8492, 7995)
[@debug] mask_train.shape = (8492, 7995)
[@debug] im.dtype = uint8
[@debug] mask_train.dtype = uint8
[@debug] im_train.shape = (12, 8492, 7995)
Train masked unique/count [ 0  1  2  3  4  5  6  7  8  9 10 11] [776057832   9399025   4170646   6930141    103494     346

ic| self.paramsTrain.path / 'full_ims/full_ims_test.npy': PosixPath('../../../dataset/dataset/cv_data/full_ims/full_ims_test.npy')


## Extract coords of image patches

In [10]:

if paramsTrain.coordsExtract == True:
  patchExtractor.extract()

del patchExtractor


STARTED PATCH EXTRACTION
[@debug] gridx.shape = (250,)
[@debug] gridy.shape = (266,)


ic| coords_train.shape: (4983, 2), coords_test.shape: (4626, 2)
ic| coords_train.dtype: dtype('int64')
ic| coords_train[0]: array([7568,  656])


## Train and evaluate


In [ ]:

model_name_id = 'model_best_' + paramsTrain.model_type + '_' + \
    paramsTrain.seq_date + '_' + paramsTrain.dataset + '_' + \
    paramsTrain.model_name + '.h5'

if paramsTrain.train == True:
  trainTest.trainAndEvaluate(model_name_id)

ic| self.model_name: 'model_best_UUnet4ConvLSTM_jun_cv_nomode.h5'


Initializing object...
12 2
[@debug] self.channel_n = 2
[@debug] self.t_len = 12
Initializing Dataset instance


ic| os.path.dirname(os.path.abspath(__file__)): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'
ic| os.getcwd(): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'
ic| self.patches['train']['coords'].shape: (4983, 2)
ic| np.unique(self.full_label_train, return_counts=True): (array([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11], dtype=uint8),
                                                           array([64671486,   710232,  1374508,    31550,   136800,   212701,
                                                                    71804,   617076,      699,    66684]))
ic| np.unique(self.full_label_test, return_counts=True): (array([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11], dtype=uint8),
                                                          array([65000618,   730790,  1199149,    25986,    88785,   312353,
                                                                   83385,   362633,      308,    89533]))
ic|

[@debug] np.unique(self.full_label_train, return_counts=True) = (array([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11], dtype=uint8), array([64671486,   710232,  1374508,    31550,   136800,   212701,
          71804,   617076,      699,    66684]))


ic| self.labels2new_labels: {0: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7, 10: 8, 11: 9}
    self.new_labels2labels: {0: 0, 1: 2, 2: 3, 3: 4, 4: 6, 5: 7, 6: 8, 7: 9, 8: 10, 9: 11}


...

393/393 [==============================] - 30s 76ms/step - loss: 0.1362 - accuracy: 0.1765 - val_loss: 0.0942 - val_accuracy: 0.2184


ic| n_batches: 46
ic| n_batches: 46


 — val_f1: [73.86 90.04 21.42 69.09 64.6  92.36 74.49  0.   56.62]
 — val_precision: [69.04 89.09 26.75 74.33 73.37 91.06 81.67  0.   40.44]
 — val_recall: [79.4  91.02 17.86 64.54 57.7  93.69 68.47  0.   94.41]
 — mean_f1: 60.275555555555556
oa 78.98
Found best weights at epoch 3


ic| n_batches: 393


Epoch 4/70
382/393 [============================>.] - ETA: 0s - loss: 0.1123 - accuracy: 0.1533